In [96]:
import malariagen_data
import pandas as pd
import numpy as np
from pyprojroot import here
from myst_nb import glue
import yaml

import bokeh.layouts as bklay
import bokeh.plotting as bkplt
import plotly.express as px

In [125]:
cohort_id = 'BF-09_Houet_colu_2012_Q3'
cohorts_analysis="20230223"
contigs = ['2L']
sample_sets = "AG1000G-BF-A"
max_cohort_size = 50
h12_calibration_contig = '3L'
sample_query = f"cohort_admin2_quarter == '{cohort_id}'"

In [98]:
# load cohorts to find sample query 
df_cohorts = pd.read_csv(here() / "build" / "final_cohorts.csv").set_index("cohort_id")
cohort = df_cohorts.loc[cohort_id]
glue("cohort_label", cohort['cohort_label']

cohort

cohort_size                                                       78
country                                                 Burkina Faso
admin1_iso                                                     BF-09
admin1_name                                            Hauts-Bassins
admin2_name                                                    Houet
taxon                                                       coluzzii
year                                                            2012
quarter                                                            3
cohort_label             Burkina Faso / Houet / coluzzii / 2012 / Q3
sample_query       cohort_admin2_quarter == 'BF-09_Houet_colu_201...
latitude                                                   11.223768
longitude                                                   -4.35639
h12_window_size                                                 1000
Name: BF-09_Houet_colu_2012_Q3, dtype: object

### {glue:}`cohort_label`

In [99]:
ag3 = malariagen_data.Ag3(
    # TODO in production build, remove use of simplecache if running inside google cloud
    # url = "gs://vo_agam_release",
    url="simplecache::gs://vo_agam_release",
    # pin the version of the cohorts analysis for reproducibility
    cohorts_analysis=cohorts_analysis,
    # TODO remove simplecache config in production
    simplecache=dict(cache_storage=(here() / "gcs_cache").as_posix()),
    results_cache=(here() / "malariagen_data_cache").as_posix(),
)
ag3

<MalariaGEN Ag3 API client>
Storage URL             : simplecache::gs://vo_agam_release
Data releases available : 3.0
Results cache           : /home/sanj/projects/selection-atlas/malariagen_data_cache
Cohorts analysis        : 20230223
Species analysis        : aim_20220528
Site filters analysis   : dt_20200416
Software version        : malariagen_data 7.3.1.post31+0b7475c
Client location         : Scotland, GB
---
Please note that data are subject to terms of use,
for more information see https://www.malariagen.net/data
or contact data@malariagen.net. For API documentation see 
https://malariagen.github.io/vector-data/ag3/api.html

In [100]:
df_samples = ag3.sample_metadata()
df_samples = df_samples.query(sample_query)

Load sample metadata:   0%|          | 0/28 [00:00<?, ?it/s]

### Map of collection sties

In [121]:
import pandas as pd
from ipyleaflet import Map, Marker, basemaps, AwesomeIcon, Popup
from ipywidgets import HTML

center = cohort[['latitude', 'longitude']].to_list()
m = Map(center=center, zoom=9, basemap=basemaps.OpenTopoMap)

for coh_id, row in df_samples.iterrows():
    lat, long = row[['latitude', 'longitude']]
    
    if row['taxon'] == 'gambiae':
        color= 'red'
    elif row['taxon'] == 'coluzzii':
        color='cadetblue'
    elif row['taxon'] == 'arabiensis':
        color='lightgreen'
    else: 
        color='gray'
    
    marker = Marker(location=(lat, long), draggable=True, opacity=0.7, color=color)
    m.add_layer(marker);
    
    # message2 = HTML()
    # message2.value = f'<a href="https://github.com/anopheles-genomic-surveillance/selection-atlas">{coh_id}</a>'
    # marker.popup = message2

display(m)

Map(center=[11.223768292682928, -4.35639024390244], controls=(ZoomControl(options=['position', 'zoom_in_text',…

### Collection dates

In [105]:
def add_empty_months(df, cohort_id):
    quarter = cohort_id[-2:]
    q_months = {'Q1':[1,2,3],
            'Q2':[4,5,6],
            'Q3':[7,8,9],
            'Q4':[10,11,12]}

    year = df.year.unique()[0]
    months = df['month'].tolist()
    empty_months = list(set(q_months[quarter]) - set(months))
    
    if empty_months:
        for m in empty_months:
            df = df.append({'year': year, 'month': m, 'count': 0}, ignore_index=True)

    return(df, quarter, year)

In [106]:
df_collection_dates = df_samples.groupby(['year', 'month']).size().reset_index().rename(columns={0: 'count'})
df_collection_dates, quarter, year = add_empty_months(df_collection_dates, cohort_id)
df_collection_dates['month'] = pd.to_datetime(df_collection_dates['month'], format='%m').dt.month_name().str.slice(stop=3)
df_collection_dates

,year,month,count
0,2012,Jul,82
1,2012,Aug,0
2,2012,Sep,0


In [107]:
px.bar(df_collection_dates, x='month', y='count', title=f"Collection dates {quarter} {year}")

In [32]:
# load window sizes 
calibration_dir = "build/h12-calibration"
with open(here() / calibration_dir / f"{cohort_id}.yaml") as calibration_file:
    calibration_params = yaml.safe_load(calibration_file)
window_size = calibration_params["h12_window_size"]

if cohort.taxon == 'arabiensis':
    phasing_analysis = 'arab'
else:
    phasing_analysis = 'gamb_colu'

if cohort.cohort_size > max_cohort_size:
    # downsampling for computational efficiency
    cohort_size = max_cohort_size
else:
    # no downsampling
    cohort_size = None 


def plot_h12_ihs_tracks(
        contig, 
        window_size, 
        phasing_analysis, 
        sample_sets, 
        sample_query, 
        cohort_size, 
        sizing_mode='stretch_width', 
        show=False, 
        width=800, 
        genes_height=100
    ):

    fig1 = ag3.plot_h12_gwss_track(
        contig=contig, 
        window_size=window_size, 
        analysis=phasing_analysis, 
        sample_sets=sample_sets,
        sample_query=sample_query, 
        cohort_size=cohort_size,
        sizing_mode=sizing_mode,
        show=show,
        width=width,
    )
    fig1.xaxis.visible = False

    fig2 = ag3.plot_h12_gwss_track(
        contig=contig, 
        window_size=window_size, 
        analysis=phasing_analysis, 
        sample_sets=sample_sets,
        sample_query=sample_query, 
        cohort_size=cohort_size,
        sizing_mode=sizing_mode,
        width=width,
        show=show,
        title="",
        x_range=fig1.x_range,
    )
    fig2.xaxis.visible = False

    fig3 = ag3.plot_genes(
        region=contig, 
        show=show,
        sizing_mode=sizing_mode,
        width=width,
        height=genes_height,
        x_range=fig1.x_range
        )
                        
    fig = bklay.gridplot(
        [fig1, fig2, fig3],
        ncols=1,
        toolbar_location="above",
        merge_tools=True,
        sizing_mode=sizing_mode,
    )

    bkplt.show(fig)


## Selection scans


### Chromosome 2RL

In [33]:
plot_h12_ihs_tracks(
    contig='2RL',
    window_size=window_size,
    phasing_analysis=phasing_analysis,
    sample_sets=sample_sets,
    sample_query=sample_query,
    cohort_size=cohort_size)

Load haplotypes:   0%|          | 0/756 [00:00<?, ?it/s]

### Chromosome 3RL

In [ ]:
plot_h12_ihs_tracks(
    contig='3RL',
    window_size=window_size,
    phasing_analysis=phasing_analysis,
    sample_sets=sample_sets,
    sample_query=sample_query,
    cohort_size=cohort_size)

### Chromosome X 

In [ ]:
plot_h12_ihs_tracks(
    contig='X',
    window_size=window_size,
    phasing_analysis=phasing_analysis,
    sample_sets=sample_sets,
    sample_query=sample_query,
    cohort_size=cohort_size)

### Diagnostics

ag3.plot_h12_calibration(
    contig=h12_calibration_contig,
    analysis=phasing_analysis,
    sample_sets=sample_sets,
    sample_query=sample_query,
    cohort_size=cohort_size,
    window_sizes=window_sizes,
)